In [33]:
import pandas as pd
from py2neo import Graph, Node, Relationship, NodeMatcher

# Task2Q3: Constructing a Knowledge Graph

In [ ]:
# auth=('username', 'key')
graph = Graph('http://localhost:7474', auth=("neo4j", "lee618121"), name='neo4j')

# Load csv
df_company = pd.read_csv('../data/KnowledgeGraph/hidy.nodes.company.csv')
df_compete = pd.read_csv('../data/KnowledgeGraph/hidy.relationships.compete.csv')
df_cooperate = pd.read_csv('../data/KnowledgeGraph/hidy.relationships.cooperate.csv')
df_dispute = pd.read_csv('../data/KnowledgeGraph/hidy.relationships.dispute.csv')
df_same_industry = pd.read_csv('../data/KnowledgeGraph/hidy.relationships.same_industry.csv')
df_invest = pd.read_csv('../data/KnowledgeGraph/hidy.relationships.invest.csv')
df_supply = pd.read_csv('../data/KnowledgeGraph/hidy.relationships.supply.csv')

# Define function to create nodes
def create_node(name, id, code, label):
    node = Node("Company", name=name, id=id, code=code, label=label)
    graph.create(node)


# Loop to create nodes for every company in df
for row in df_company.itertuples(index=False):
    id = row[0]
    name = row[1]
    code = row[2]
    label = row[3]

    create_node(name, id, code, label)

# Create relationship
for row in df_compete.itertuples(index=False):
    start_id = row[0]
    end_id = row[1]
    relation_type = row[2]
    start_time = row[3]

    start_node = graph.nodes.match("Company", id=start_id).first()
    end_node = graph.nodes.match("Company", id=end_id).first()

    if start_node and end_node:
        relationship = Relationship(start_node, relation_type, end_node, start_time=start_time)
        graph.create(relationship)

        reverse_relationship = Relationship(end_node, relation_type, start_node, start_time=start_time)
        graph.create(reverse_relationship)


# Create relationship
for row in df_cooperate.itertuples(index=False):
    start_id = row[0]
    end_id = row[1]
    relation_type = row[2]
    start_time = row[3]

    start_node = graph.nodes.match("Company", id=start_id).first()
    end_node = graph.nodes.match("Company", id=end_id).first()

    if start_node and end_node:
        relationship = Relationship(start_node, relation_type, end_node, start_time=start_time)
        graph.create(relationship)

        reverse_relationship = Relationship(end_node, relation_type, start_node, start_time=start_time)
        graph.create(reverse_relationship)


# Create relationship
for row in df_dispute.itertuples(index=False):
    start_id = row[0]
    end_id = row[1]
    relation_type = row[2]
    start_time = row[3]

    start_node = graph.nodes.match("Company", id=start_id).first()
    end_node = graph.nodes.match("Company", id=end_id).first()

    if start_node and end_node:
        relationship = Relationship(start_node, relation_type, end_node, start_time=start_time)
        graph.create(relationship)

        reverse_relationship = Relationship(end_node, relation_type, start_node, start_time=start_time)
        graph.create(reverse_relationship)


# Create relationship
for row in df_same_industry.itertuples(index=False):
    start_id = row[0]
    end_id = row[1]
    relation_type = row[2]
    start_time = row[3]

    start_node = graph.nodes.match("Company", id=start_id).first()
    end_node = graph.nodes.match("Company", id=end_id).first()

    if start_node and end_node:
        relationship = Relationship(start_node, relation_type, end_node, start_time=start_time)
        graph.create(relationship)

        reverse_relationship = Relationship(end_node, relation_type, start_node, start_time=start_time)
        graph.create(reverse_relationship)


# Create relationship
for row in df_invest.itertuples(index=False):
    start_id = row[0]
    end_id = row[1]
    relation_type = row[2]

    start_node = graph.nodes.match("Company", id=start_id).first()
    end_node = graph.nodes.match("Company", id=end_id).first()

    if start_node and end_node:
        relationship = Relationship(start_node, relation_type, end_node)
        graph.create(relationship)


# Create relationship
for row in df_supply.itertuples(index=False):
    start_id = row[0]
    end_id = row[1]
    relation_type = row[2]

    start_node = graph.nodes.match("Company", id=start_id).first()
    end_node = graph.nodes.match("Company", id=end_id).first()

    if start_node and end_node:
        relationship = Relationship(start_node, relation_type, end_node)
        graph.create(relationship)

# Task2Q4: Knowledge-Driven Financial Analysis

In [34]:
# Load EXCEL from task one
df = pd.read_excel('../result/Task1.xlsx')

In [51]:
# Define a function to get Implicit_Positive_Company
def get_company_pos(explicit_company):
        pos_company_list = []
        explicit_company_list = explicit_company.split(',')
        for i in explicit_company_list:
            query_pos = f"MATCH (c1:Company)-[:same_industry|supply|invest|cooperate]->(c2:Company) WHERE c1.name = '{i}'  RETURN DISTINCT c2.name UNION ALL MATCH (c1:Company)<-[:supply|invest]-(c2:Company) WHERE c1.name = '{i}' RETURN DISTINCT c2.name"
            results_pos = graph.run(query_pos).data()
            for j in results_pos:
                pos_company_list.append(j['c2.name'])
        
        uni_pos_company_list = list(dict.fromkeys(pos_company_list))  
        return ','.join(uni_pos_company_list) if uni_pos_company_list else None

# Define a function to get Implicit_Negative_Company
def get_company_neg(explicit_company):
        neg_company_list = []
        explicit_company_list = explicit_company.split(',')
        for i in explicit_company_list:
            query_neg = f"MATCH (c1:Company)-[:compete|dispute]->(c2:Company) WHERE c1.name = '{i}' RETURN DISTINCT c2.name"
            results_neg = graph.run(query_neg).data()
            for j in results_neg:
                neg_company_list.append(j['c2.name'])
        
        uni_neg_company_list = list(dict.fromkeys(neg_company_list))  
        return ','.join(uni_neg_company_list) if uni_neg_company_list else None

In [57]:
# Apply the function to Explicit_Company
df['Implicit_Positive_Company'] = df['Explicit_Company'].apply(get_company_pos)
df['Implicit_Negative_Company'] = df['Explicit_Company'].apply(get_company_neg)

In [58]:
# Check
df

,NewsID,NewsContent,Explicit_Company,label,Implicit_Positive_Company,Implicit_Negative_Company
0,1,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,建设银行,0,"兴业银行,晨鸣纸业,中国建筑,广电运通,中国石化,中国人寿,京东方,中国船舶,农业银行,中金...","任子行,捷捷微电"
1,2,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,"中金公司,农业银行",1,"上汽集团,中国电信,中信证券,宁德时代,中国船舶,工商银行,中国太保,东方财富,浙商银行,中...","交控科技,ST云维"
2,3,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,"外运发展,中国国航",1,"中国外运,南方航空,寒武纪,农业银行,中国电建,兴业证券,中国石化,山航B,中国交建,工商银...",None
3,4,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,胜利股份,1,"新疆浩源,特锐德",None
4,5,全景网11月30日讯 外围股市造好，带动港股今早造好，恒指高开后反覆上升，最高升252点...,"*ST高升,交通银行,建设银行,中国银行,工商银行",1,"中国铁建,中国人寿,农业银行,中国银行,中国石油,连云港,风华高科,建设银行,申通地铁,宝丰...","*ST长生,中信国安,中国银行,华发股份,任子行,捷捷微电,山东黄金,中国平安,中国人保,海..."
...,...,...,...,...,...,...
495144,1037031,每经AI快讯，有投资者在投资者互动平台提问：请问公司目前有没有电解槽产能，规划情况能否详细介...,亿华通,1,"宝泰隆,福田汽车,东风汽车,百奥泰,仕佳光子,中国船舶,飞龙股份,东旭光电",None
495145,1037032,依米康（SZ 300249，收盘价：10.38元）发布公告称，2023年10月12日，依米康...,"中泰证券,依米康",1,"京东方,招商证券,中国银行,兴业证券,国泰君安,东方证券,智飞生物,东北证券,申万宏源,光大...","华谊嘉信,西水股份"
495146,1037033,天风证券10月13日发布研报称，给予中核科技（000777.SZ，最新价：13.03元）买入...,"中核科技,天风证券",1,"中国核电,东吴证券,招商银行,长江证券,中信证券,恒生电子,凯撒文化,东北证券,方正证券,西...","吉翔股份,中源家居,三特索道"
495147,1037034,有投资者提问：抗癌药CPT获批后，公司是否应该按照股权协议继续收购沙东股权，适应症为MM的C...,海特生物,1,"药明康德,海尔生物",None


In [59]:
# Save
df.to_excel('../result/Task2.xlsx', encoding='utf-8', index=False)

C:\Users\Lee\miniconda3\envs\p5002\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
